In [ ]:
from comparer import *
from git import *
from main import *
from util import localfile_tool

repo = 'angular/angular.js'
num1 = '6341'
num2 = '6167'

m = classify()

with open('data/msr_positive_pairs.txt') as f:
    pairs = f.readlines()

label = 1

num_ok, num_total = 0, 0

X, y = [], []

for pair in pairs:
    repo, num1, num2 = pair.split()
    
    p1 = get_pull(repo, num1)
    p2 = get_pull(repo, num2)
    
    '''
    if (p1["merged_at"] is not None) and (p2["merged_at"] is not None):
        continue
    '''
    
    cl1 = get_pull_commit(p1)
    cl2 = get_pull_commit(p2)
    
    '''
    if (len(cl1) == 1) and (len(cl2) == 1):
        continue
    '''
    
    if (len(cl1) >= 100) or (len(cl2) >= 100):
        continue
    if check_too_big(p1) or check_too_big(p2):
        continue
    
    max_s, max_p1, max_p2, max_vet = -1, None, None, None
    
    for c1 in cl1:
        for c2 in cl2:
            if c1['url'] != c2['url']:
                vet = get_sim_vector_on_commit(c1, c2)
                t = m.predict_proba([vet])[0][1]
                if t > max_s:
                    max_s, max_p1, max_p2, max_vet = t, c1['html_url'], c2['html_url'], vet
    
    X.append(vet)
    y.append(label)
    
    num_total += 1
    if num_total % 100 == 0:
        print(num_total)
    
    
    '''
    if m.predict([get_sim_vector(p1, p2)])[0] == 1:
        ok = True
    '''
    # print(m.predict_proba([get_sim_vector(p1, p2)])[0][1])
    '''
    if max_s >= 0.8:
        print(max_s, max_p1, max_p2, p1['html_url'] + '/commits', p2['html_url'] + '/commits', sep='\n')
        print(len(cl1), len(cl2))
    '''
    
    '''
    if ok:
        num_ok += 1
    num_total += 1
    if num_total >= 500:
        break
    print(num_total)
    '''

    
# print(1.0 * num_ok / num_total)
    

out = 'data/msr_positive_pairs_feature_vector_commit_g'

X_path = out + '_X.json'
y_path = out + '_y.json'

localfile_tool.write_to_file(X_path, X)
localfile_tool.write_to_file(y_path, y)
    